# Welcome to DeepCheck for evaluating macro genome assemblies!
There are three steps to use DeepCheck. 

##the first step generates features for DeepCheck input!



In [ ]:
import pickle
from os import listdir
import pandas as pd
from scipy.sparse import csr_matrix, vstack
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse import coo_matrix, vstack, hstack
from scipy import sparse
import warnings
import process_data
import torch
import numpy as np
from scipy.sparse import load_npz
from model import ResidualBlock, ResNet
import os
#读取pkl文件
import pickle
#读取pkl文件
import pandas as pd
#读取pkl文件
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [ ]:
# 在Jupyter Notebook中运行以下代码
input_folder = "path/to/your/folder_with_bins"  
output_folder = "path/to/your/output_folder"

# 调用Python脚本
!python generate_features.py {input_folder} {output_folder}


input_folder: You need to predict the sequence of bin's fasta
output_folder :The output of the feature is a pkl file

In [ ]:
fasta_names=[]  #
fast_features_matrix=process_data.read_feature(fasta_names)

#Call process_data.read_feature to read the feature matrix for loading the model

In [ ]:

def predict_completeness_and_contamination(fast_features_matrix, model_path, device):
    # Load the model
    model = ResNet(ResidualBlock, [2, 2, 2, 2])
    model.load_state_dict(torch.load(model_path))
    model.to(device)
    model.eval()

    # Preprocess the input data
    zero_padded_features = np.zeros((fast_features_matrix.shape[0], 20164))
    zero_padded_features[:, :20021] = fast_features_matrix
    image_shape = (142, 142)

    # Convert to PyTorch tensor
    inputs = torch.Tensor(zero_padded_features).reshape(-1, 1, *image_shape).to(device)

    # Make predictions
    with torch.no_grad():
        outputs_comp, outputs_cont = model(inputs)

    # Convert predictions to numpy arrays
    predictions_comp = outputs_comp.cpu().numpy() * 100
    predictions_cont = outputs_cont.cpu().numpy() * 100

    return predictions_comp, predictions_cont

# load the model

In [ ]:

model_path = 'models/best_model.pt'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



# #Predicting contamination and completeness

In [ ]:
predictions_comp, predictions_cont = predict_completeness_and_contamination(fast_features_matrix, model_path, device)

print("Predicted Completeness:", predictions_comp)
print("Predicted Contamination:", predictions_cont)